# Deep Sleep Net

In [1]:
# Imports
import cnn_utilities as cnn_utils
from cnn_utilities import *
import utilities as utils
from utilities import *
import seaborn as sns
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 160
colors = sns.color_palette("viridis", 10)

In [2]:
# Parameters
data_dir = "../data/preprocessed"
data_source_type = "gdansk" # ["gdansk", "physionet"]
splice_type = "constant" # ["complete", "constant", "random"]
label_type = "regression" # ["classification", "regression"]
simulation = True # [True, False]
class_weights = False # [True, False]
sampling = "oversample" # ["none", "undersample", "oversample"]

batch_size = 8
no_workers = 1
double_precision = False # Only for regression

epochs = 120
learning_rate = 0.000001

In [3]:
# Auto adjustment
data_type = "simulated" if simulation else "original"
class_weights_type = "weighted_" if class_weights else "unweighted_"

if label_type == "regression":
    class_weights_type = ""

if simulation:
    N = 96
    simulated = 100
    data_source_type = "gdansk"
    splice_type = "constant"
    epochs //= 10
else:
    N = 48
    simulated = None

read_path = f"{data_dir}/{data_type}_{data_source_type}_{splice_type}_deep_{label_type}_seconds_"
y_name = "label" if label_type == "classification" else "age"
no_classes = 7 if data_source_type == "gdansk" else 6
N = 48 if data_source_type == "gdansk" else 240
pad_length = 27_000 if data_source_type == "gdansk" else 135_000
side_arm_mlp = 128 if data_source_type == "gdansk" else 64
channels = 128 if data_source_type == "gdansk" else 64
lstm_hidden = 256 if data_source_type == "gdansk" else 128
side_arm_mlp = 32 if data_source_type == "physionet" and splice_type=="complete" else side_arm_mlp

if splice_type == "constant":
    pad_length //= N
    batch_size *= 64
    
if data_source_type == "physionet" and splice_type == "complete":
    batch_size //= 8
    
if label_type == "classification" and splice_type == "complete":
    batch_size //= 8
    
if label_type == "classification" or data_source_type == "physionet":
    epochs //= 3

batch_size = max(1, batch_size)

classification = label_type=="classification"
complete = splice_type == "complete"
truncate = None

if simulation:
    pad_length = 200
    truncate = pad_length

if sampling == "oversample":
    sample_addon = "oversample"
elif sampling == "undersample":
    sample_addon = "undersample"
else:
    sample_addon = "none"

relative_folder_dir = "../report/img/learning/"
basic_path = f"{data_type}_{data_source_type}_sleepnet_{label_type}_{splice_type}_{sample_addon}_{class_weights_type}"
model_save_path = "models/" + basic_path + f"model.save"
loss_save_path = relative_folder_dir + basic_path + "loss.png"
accuracy_save_path = relative_folder_dir + basic_path + "accuracy.png"
error_distribution_save_path = relative_folder_dir + basic_path + "error_distribution.png"
error_distribution_unbiased_save_path = relative_folder_dir + basic_path + "error_distribution_unbiased.png"

metrics_save_path = f"../report/results/{basic_path}metrics.txt"

In [4]:
metrics_save_path

'../report/results/simulated_gdansk_sleepnet_regression_constant_oversample_metrics.txt'

In [5]:
model_save_path

'models/simulated_gdansk_sleepnet_regression_constant_oversample_model.save'

In [6]:
epochs

12

# Data

In [7]:
%%time
if sampling == "undersample":
    if not os.path.exists(read_path + "train_undersample.csv"):
        print(f"Creating undersampled dataset '{read_path}train_undersample.csv'.")
        train = pd.read_csv(read_path + "train.csv", index_col=0)
        train_undersampled = utils.undersample_df(train, y_name)
        train_undersampled.to_csv(read_path + "train_undersample.csv", index_label=None)
        del train, train_undersampled
    else:
        print(f"Undersampled dataset '{read_path}train_undersample.csv' does already exist.")
    sample_addon = "_undersample"
elif sampling == "oversample":
    if not os.path.exists(read_path + "train_oversample.csv"):
        print(f"Creating oversampled dataset '{read_path}train_oversample.csv'.")
        train = pd.read_csv(read_path + "train.csv", index_col=0)
        train_oversampled = utils.oversample_df(train, y_name)
        train_oversampled.to_csv(read_path + "train_oversample.csv", index_label=None)
        del train, train_oversampled
    else:
        print(f"Oversampled dataset '{read_path}train_oversample.csv' does already exist.")
    sample_addon = "_oversample"
else:
    print(f"Taking dataset '{read_path}train.csv' without sampling.")
    sample_addon = ""

Creating oversampled dataset '../data/preprocessed/simulated_gdansk_constant_deep_regression_seconds_train_oversample.csv'.


/home/flennic/anaconda3/envs/master-thesis/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 8min 27s, sys: 13.5 s, total: 8min 40s
Wall time: 9min 6s


In [8]:
%%time
train, train_data_set = cnn_utils.path_to_DataLoader(read_path + f"train{sample_addon}.csv", index=True, classification=classification, batch_size=batch_size)
val, val_data_set = cnn_utils.path_to_DataLoader(read_path + "val.csv", index=True, classification=classification, batch_size=batch_size, truncate=truncate)
test, test_data_set = cnn_utils.path_to_DataLoader(read_path + "test.csv", index=True, classification=classification, batch_size=batch_size, truncate=truncate)

CPU times: user 4min 10s, sys: 48.2 s, total: 4min 58s
Wall time: 8min 5s


## Model

In [9]:
if label_type == "classification":
    class_weights = torch.tensor(utils.class_weights_from_path(read_path + f"train{sample_addon}.csv", no_classes, y_name), dtype=torch.float).cuda() if class_weights else None
    print(class_weights)
    model = DeepSleepNet(no_classes,
                         data_length=pad_length,
                         batch_size=batch_size,
                         complete=complete,
                         side_arm_mlp=side_arm_mlp,
                         channels=channels,
                         lstm_hidden=lstm_hidden).cuda()
elif label_type == "regression":
    model = DeepSleepNet(no_classes,
                         data_length=pad_length,
                         batch_size=batch_size,
                         classification=classification,
                         side_arm_mlp=side_arm_mlp,
                         channels=channels,
                         lstm_hidden=lstm_hidden).cuda()
else:
    print("label_type not supported.")
    
if double_precision:
    model.double()
    
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
if classification:
    criterion = torch.nn.NLLLoss(weight = class_weights)
else:
    criterion = torch.nn.MSELoss()

## Training

In [10]:
model_save_path

'models/simulated_gdansk_sleepnet_regression_constant_oversample_model.save'

In [ ]:
%%time

training_loss_storage = []
training_accuracy_storage = []
validation_loss_storage = []
validation_accuracy_storage = []

for i in range(epochs):
    
    print("\nEpoch: {}".format(i+1))
    
    training_loss = 0
    training_accuracy = 0
    training_processed_data = 0
    training_processed_batches = 0
    
    for x, y in train:
        
        # Initialize hidden states in each batch
        model.init_hidden(x.shape[0])
        
        x = x.cuda()
        y = y.cuda()
        
        # Reset Gradients
        optimizer.zero_grad()

        # Forward, Loss, Backwards, Update
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        training_processed_data += x.shape[0]
        training_processed_batches += 1

        # Calculate Metrics
        training_loss += loss.item()
        
        if classification:
            training_accuracy += torch.sum(torch.exp(output).topk(1)[1].view(-1) == y).item()
        else:
            training_accuracy += torch.sum(torch.abs(output - y) < 5).cpu().numpy()
        
        print(f"{training_processed_data}/{len(train_data_set)} ({round(training_processed_data/len(train_data_set)*100, 2)}%)", end="\r")
    
    else:
        print("Training Mean Absolute Error: {}".format(np.sqrt(training_loss/training_processed_batches)))
        print("Training Loss: {}".format(training_loss/training_processed_batches))
        print("Training Accuracy: {}".format(training_accuracy/training_processed_data))
        
        training_loss_storage.append(training_loss/training_processed_batches)
        training_accuracy_storage.append(training_accuracy/training_processed_data)
        
        validation_loss = 0
        validation_accuracy = 0
        validation_processed_data = 0
        validation_processed_batches = 0

        model.eval()

        with torch.no_grad():
            for x, y in val:
                
                model.init_hidden(x.shape[0])
                
                x = x.cuda()
                y = y.cuda()

                output_validation = model(x)
                loss_val = criterion(output_validation, y)
                
                validation_processed_data += x.shape[0]
                validation_processed_batches += 1
                
                validation_loss += loss_val.item()
                
                if classification:
                    validation_accuracy += torch.sum(
                        torch.exp(output_validation).topk(1, dim=1)[1].view(-1) == y).item()
                else:
                    validation_accuracy += torch.sum(torch.abs(output_validation - y) < 5).cpu().numpy()
                
            else:
                print("Validation Mean Absolute Error: {}".format(np.sqrt(validation_loss/validation_processed_batches)))
                print("Validation Loss: {}".format(validation_loss/validation_processed_batches))
                print("Validation Accuracy: {}".format(validation_accuracy/validation_processed_data))
                
                # Save model if the validation accuracy was the lowest so far.
                #if all((validation_accuracy/validation_processed_data) >= np.array(validation_accuracy_storage)) or len(validation_accuracy_storage) == 0:
                #    torch.save(model.state_dict(), model_save_path)
                #    print(f"Model saved. Epoch {i+1}.")
                    
                # Save model if the validation loss was the lowest so far.
                if all((validation_loss/validation_processed_batches) <= np.array(validation_loss_storage)) or len(validation_loss_storage) == 0:
                    torch.save(model.state_dict(), model_save_path)
                    print(f"Model saved. Epoch {i+1}.")
                
                validation_loss_storage.append(validation_loss/validation_processed_batches)
                validation_accuracy_storage.append(validation_accuracy/validation_processed_data)
                    
                model.train()



Epoch: 1
Training Mean Absolute Error: 36.0236291299416
Training Loss: 1297.7018556915768
Training Accuracy: 0.10906684027777777
Validation Mean Absolute Error: 19.778156686265458
Validation Loss: 391.17548190646704
Validation Accuracy: 0.13910011574074074
Model saved. Epoch 1.

Epoch: 2
Training Mean Absolute Error: 17.244543423959268
Training Loss: 297.3742779008169
Training Accuracy: 0.19668712797619048
Validation Mean Absolute Error: 21.432904168672888
Validation Loss: 459.36938110351565
Validation Accuracy: 0.15694733796296295

Epoch: 3
Training Mean Absolute Error: 16.790471855818115
Training Loss: 281.91994514102026
Training Accuracy: 0.20469308035714287
Validation Mean Absolute Error: 19.29045579076668
Validation Loss: 372.12168461552375
Validation Accuracy: 0.1533101851851852
Model saved. Epoch 3.

Epoch: 4
Training Mean Absolute Error: 16.50968257578262
Training Loss: 272.5696187531002
Training Accuracy: 0.2127064732142857
Validation Mean Absolute Error: 19.6797197927429
Val

In [ ]:
sns.lineplot(range(len(training_loss_storage)), training_loss_storage, label="Training Loss", color=colors[9]);
sns.lineplot(range(len(validation_loss_storage)), validation_loss_storage, label="Validation Loss", color=colors[0]);
plt.legend()
plt.ylabel('Loss');
plt.xlabel('Epoch');
plt.title(f'Loss During Training ({label_type} | {splice_type})');
plt.savefig(loss_save_path);
plt.show();

In [ ]:
sns.lineplot(range(len(training_accuracy_storage)), training_accuracy_storage, label="Training Accuracy", color=colors[9]);
sns.lineplot(range(len(validation_accuracy_storage)), validation_accuracy_storage, label="Validation Accuracy", color=colors[0]);
plt.legend()
plt.ylabel('Accuracy');
plt.xlabel('Epoch');
plt.title(f'Accuracy During Training ({label_type} | {splice_type})');
plt.savefig(accuracy_save_path);
plt.show();

## Evaluation

In [ ]:
del train, val, test, train_data_set, val_data_set, test_data_set

In [ ]:
%%time

result = ""

model.load_state_dict(torch.load(model_save_path))

if label_type == "classification" and splice_type == "complete":
    train_res, train_error_distribution = cnn_utils.evaluate_classification_complete(model, read_path + "train.csv", batch_size=batch_size, no_classes=no_classes)
    val_res, val_error_distribution = cnn_utils.evaluate_classification_complete(model, read_path + "val.csv", batch_size=batch_size, no_classes=no_classes)
    test_res, test_error_distribution = cnn_utils.evaluate_classification_complete(model, read_path + "test.csv", batch_size=batch_size, no_classes=no_classes)

elif label_type == "regression" and splice_type == "complete":
    train_res, train_error_distribution = cnn_utils.evaluate_regression_complete(model, read_path + "train.csv", batch_size=batch_size, no_classes=no_classes)
    val_res, val_error_distribution = cnn_utils.evaluate_regression_complete(model, read_path + "val.csv", batch_size=batch_size, no_classes=no_classes)
    test_res, test_error_distribution = cnn_utils.evaluate_regression_complete(model, read_path + "test.csv", batch_size=batch_size, no_classes=no_classes)
    
    train_res_unbiased, _ = cnn_utils.evaluate_regression_complete(model,read_path + "train.csv", batch_size=batch_size, no_classes=no_classes,
                                                            bias=np.mean(val_error_distribution))
    val_res_unbiased, _ = cnn_utils.evaluate_regression_complete(model,read_path + "val.csv", batch_size=batch_size, no_classes=no_classes,
                                                            bias=np.mean(val_error_distribution))
    test_res_unbiased, test_unbiased_error_distribution = cnn_utils.evaluate_regression_complete(model,read_path + "test.csv", batch_size=batch_size,
                                                            no_classes=no_classes,
                                                            bias=np.mean(val_error_distribution))

elif label_type == "classification" and splice_type == "constant":
    train_res, train_error_distribution = cnn_utils.evaluate_classification_constant(model, read_path + "train.csv", batch_size=batch_size, N=N, no_classes=no_classes, simulated=simulated)
    val_res, val_error_distribution = cnn_utils.evaluate_classification_constant(model, read_path + "val.csv", batch_size=batch_size, N=N, no_classes=no_classes, simulated=simulated)
    test_res, test_error_distribution = cnn_utils.evaluate_classification_constant(model, read_path + "test.csv", batch_size=batch_size, N=N, no_classes=no_classes, simulated=simulated)

elif label_type == "regression" and splice_type == "constant":
    train_res, train_error_distribution = cnn_utils.evaluate_regression_constant(model, read_path + "train.csv", batch_size=batch_size, N=N, no_classes=no_classes, simulated=simulated)
    val_res, val_error_distribution = cnn_utils.evaluate_regression_constant(model, read_path + "val.csv", batch_size=batch_size, N=N, no_classes=no_classes, simulated=simulated)
    test_res, test_error_distribution = cnn_utils.evaluate_regression_constant(model, read_path + "test.csv", batch_size=batch_size, N=N, no_classes=no_classes, simulated=simulated)
    
    train_res_unbiased, _ = cnn_utils.evaluate_regression_constant(model, read_path + "train.csv", batch_size=batch_size, N=N, no_classes=no_classes,
                                                            bias=np.mean(val_error_distribution), simulated=simulated)
    val_res_unbiased, _ = cnn_utils.evaluate_regression_constant(model, read_path + "val.csv", batch_size=batch_size, N=N, no_classes=no_classes,
                                                            bias=np.mean(val_error_distribution), simulated=simulated)
    test_res_unbiased, test_unbiased_error_distribution = cnn_utils.evaluate_regression_constant(model, read_path + "test.csv", batch_size=batch_size, N=N,
                                                            no_classes=no_classes,
                                                            bias=np.mean(val_error_distribution), simulated=simulated)
    
else:
    print("label_type and splice_type combination not supported.")

result += f"Training Accuracy: {train_res}\n"
result += f"Validation Accuracy: {val_res}\n"
result += f"Test Accuracy: {test_res}\n\n"
result += f"Estimated Bias: {np.mean(val_error_distribution)}\n\n"

if not classification:
    result += f"Unbiased Training Accuracy: {train_res_unbiased}\n"
    result += f"Unbiased Validation Accuracy: {val_res_unbiased}\n"
    result += f"Unbiased Test Accuracy: {test_res_unbiased}\n\n"

In [ ]:
bins = len(np.unique(test_error_distribution)) if classification else None
sns.set(style="white")
sns.distplot(test_error_distribution, color=sns.color_palette("viridis", 10)[5], label="Error Density", bins=bins)
plt.ylabel('Density');
plt.xlabel('Error');
plt.title(f'Error Distribution ({label_type} | {splice_type})');
plt.legend();
plt.savefig(error_distribution_save_path);
plt.show();

In [ ]:
if not classification:
    sns.set(style="white")
    sns.distplot(test_unbiased_error_distribution, color=sns.color_palette("viridis", 10)[5], label="Error Density")
    plt.ylabel('Density');
    plt.xlabel('Error');
    plt.title(f'Unbiased Error Distribution ({label_type} | {splice_type})');
    plt.legend();
    plt.savefig(error_distribution_unbiased_save_path);
    plt.show();

In [ ]:
if not classification:
    unbiased_mean = np.mean(test_unbiased_error_distribution)
    unbiased_std = np.std(test_unbiased_error_distribution)
    ci = np.percentile(test_unbiased_error_distribution, [2.5, 97.5])
    
    result += "\n\n"
    result += f"# Estimates\n"
    result += f"Unbiased Mean:\t\t\t{unbiased_mean}\n"
    result += f"Standard Deviation:\t\t{unbiased_std}\n"
    result += f"95% Prediction Interval:\t{unbiased_mean - 1.96 * unbiased_std} to {unbiased_mean + 1.96 * unbiased_std}\n\n"
    result += f"# Estimate from percentiles\n"
    result += f"95% Prediction Interval:\t{ci[0]+unbiased_mean} to {ci[1]+unbiased_mean}\n\n"
    result += f"# Pure PI\n"
    result += f"95% Prediction Interval:\t{- 1.96 * unbiased_std} to {1.96 * unbiased_std}\n"
    

In [ ]:
result += f"\nEpochs: {epochs}\n"
result += f"Learning Rate: {learning_rate}\n"
result += f"Batch Size: {batch_size}\n"
result += f"Side-Arm MLP: {side_arm_mlp}\n"
result += f"LSTM Hidden: {lstm_hidden}\n"
result += f"Channels: {channels}\n"
result += f"Sampling: {sampling}"

In [ ]:
with open(metrics_save_path, "w") as text_file:
    text_file.write(result)

In [ ]:
print(result)